In [ ]:
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import torch
import torch_geometric
import torch
import os
import os.path as osp
import numpy as np
from tqdm import tqdm as tqdm
#from tqdm.notebook import tqdm

print("pyg:",torch_geometric.__version__)
print("torch:",torch.__version__)

In [ ]:
import glob
import matplotlib.pyplot as plt

In [ ]:
eb_scale=50.
m0_scale=18

In [ ]:
def transform_y(y):
    return (y)/m0_scale

def inv_transform(y):
    return (y*m0_scale)

In [ ]:
data_list_pi=[]
raw_dir= '/export/home/phys/aharilal/MassRegressor/GNNscripts/GNNDATA/Pi0_graphs/Unbiased/Withxyz/0000/'
fnamelistt = [filepath for filepath in glob.glob(raw_dir+'data*.pt')]
for i in tqdm(fnamelistt):
    idx = torch.load(i)
    idx.genmass = transform_y(idx.pmass)
    #print(idx.genmass)
    #idx.X= idx.x
    idx.x[:,0]=idx.x[:,0]/eb_scale
    data_list_pi.append(idx)


EB_list=[]
m_list=[]
for i in range(len(data_list_pi)):
    id_ = data_list_pi[i]
    #print(idx.x[:,0].shape)
    #EB_list.append(id_.x[:,0])
    m_list.append(id_.genmass)

In [ ]:
### pmass, pscieta, psciphi : tensor names
# import pho data
hi,lo = -2., 0.


import glob
raw_dir= '/export/home/phys/aharilal/MassRegressor/GNNscripts/GNNDATA/Gamma_graphs/Withxyz/0000/'#'/data_CMS/cms/sghosh/ECALGNNDATA/GRAPHS_ietaiphi/GenGamma/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data*.pt')]
data_list_pho = []
ctr = 0
for i in tqdm(fnamelist):
    idx = torch.load(i)
    idx.genmass =  torch.tensor((hi-lo)*np.random.random_sample(size=(1,1)) + lo , dtype=torch.float32)
    #print(idx.genmass)
    #idx.x[] = idx.x
    idx.x[:,0] = idx.x[:,0]/eb_scale
    idx.genmass = transform_y(idx.genmass)
    #print(idx.genmass)
    
    data_list_pho.append(idx)
    #data_list.append(torch.load(i))
    
    if ctr > 31000 :
        break
    ctr +=1
        
        
print(data_list_pho[0].pmass,data_list_pho[0].genmass)
totalevpho = len(data_list_pho)
#trainev ='''

In [ ]:
plot_dir='/export/home/phys/aharilal/MassRegressor/a2gg-regression/PLOTS/GNN_plots/GNN_scale50_residual_Wxyz_catCoord_Lrate80'
#for d in ['MODELS','PLOTS']:
if not os.path.isdir(plot_dir):
    os.makedirs(plot_dir)

checkpoint_dir = '/export/home/phys/aharilal/MassRegressor/a2gg-regression/MODELS/GNN_scale50_residual_Wxyz_catCoord_Lrate80'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
#del data_list_comb
trainfrac = 0.8

totalevpi = len(data_list_pi)
trainpi = 151000 #int(0.8*totalevpi)

trainpho = 27768 #48000 #int((trainpi/18.0)*(lo-hi))*2

print("totalevpi,trainpi,trainpho :",totalevpi,trainpi,trainpho)

import random

data_list_photest = data_list_pho  #[]
random.shuffle(data_list_pi)
data_list_train = data_list_pi[:trainpi] + data_list_photest[:trainpho]
random.shuffle(data_list_train)
totaltrain = len(data_list_train)
print("total train evs:",totaltrain)


valpi = 32895#len(data_list_pi[trainpi:])
valpho = 3086#3000##int((valpi/18.0)*(lo-hi))

data_list_val = data_list_pi[trainpi:] #+ data_list_photest[trainpho:trainpho+valpho]

print("valpi,valpho,len(data_list_val) :",valpi,valpho,len(data_list_val))


import torch_geometric
ntrainbatch = 500
ntestbatch = 500
trainloader = torch_geometric.data.DataLoader(data_list_train, batch_size=ntrainbatch)
testloader = torch_geometric.data.DataLoader(data_list_val, batch_size=ntestbatch)
#batch_size = ntrainbatch
epoch_size = len(data_list_train)
print("epoch size,batch_size:",epoch_size,ntrainbatch)

In [ ]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
import math
import torch
import sys

class ReduceMaxLROnRestart:
    def __init__(self, ratio=0.75):
        self.ratio = ratio
        
        def __call__(self, eta_min, eta_max):
            return eta_min, eta_max * self.ratio
        
        
class ExpReduceMaxLROnIteration:
    def __init__(self, gamma=1):
        self.gamma = gamma
        
    def __call__(self, eta_min, eta_max, iterations):
        return eta_min, eta_max * self.gamma ** iterations


class CosinePolicy:
    def __call__(self, t_cur, restart_period):
        return 0.5 * (1. + math.cos(math.pi *
                                    (t_cur / restart_period)))
    
    
class ArccosinePolicy:
    def __call__(self, t_cur, restart_period):
        return (math.acos(max(-1, min(1, 2 * t_cur
                                      / restart_period - 1))) / math.pi)
    
    
class TriangularPolicy:
    def __init__(self, triangular_step=0.5):
        self.triangular_step = triangular_step
        
    def __call__(self, t_cur, restart_period):
        inflection_point = self.triangular_step * restart_period
        point_of_triangle = (t_cur / inflection_point
                             if t_cur < inflection_point
                             else 1.0 - (t_cur - inflection_point)
                             / (restart_period - inflection_point))
        return point_of_triangle
    
    
class CyclicLRWithRestarts(_LRScheduler):
    """Decays learning rate with cosine annealing, normalizes weight decay
    hyperparameter value, implements restarts.
    https://arxiv.org/abs/1711.05101
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        batch_size: minibatch size
        epoch_size: training samples per epoch
        restart_period: epoch count in the first restart period
        t_mult: multiplication factor by which the next restart period will expand/shrink
        policy: ["cosine", "arccosine", "triangular", "triangular2", "exp_range"]
        min_lr: minimum allowed learning rate
        verbose: print a message on every restart
        gamma: exponent used in "exp_range" policy
        eta_on_restart_cb: callback executed on every restart, adjusts max or min lr
        eta_on_iteration_cb: callback executed on every iteration, adjusts max or min lr
        triangular_step: adjusts ratio of increasing/decreasing phases for triangular policy
    Example:
        >>> scheduler = CyclicLRWithRestarts(optimizer, 32, 1024, restart_period=5, t_mult=1.2)
        >>> for epoch in range(100):
        >>>     scheduler.step()
        >>>     train(...)
        >>>         ...
        >>>         optimizer.zero_grad()
        >>>         loss.backward()
        >>>         optimizer.step()
        >>>         scheduler.batch_step()
        >>>     validate(...)
    """
    
    def __init__(self, optimizer, batch_size, epoch_size, restart_period=100,
                 t_mult=2, last_epoch=-1, verbose=False,
                 policy="cosine", policy_fn=None, min_lr=1e-7,
                 eta_on_restart_cb=None, eta_on_iteration_cb=None,
                 gamma=1.0, triangular_step=0.5):
        
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        
        self.optimizer = optimizer
        
        if last_epoch == -1:
            for group in optimizer.param_groups:
                group.setdefault('initial_lr', group['lr'])
                group.setdefault('minimum_lr', min_lr)
        else:
            for i, group in enumerate(optimizer.param_groups):
                if 'initial_lr' not in group:
                    raise KeyError("param 'initial_lr' is not specified "
                                   "in param_groups[{}] when resuming an"
                                   " optimizer".format(i))
                
        self.base_lrs = [group['initial_lr'] for group
                         in optimizer.param_groups]
        
        self.min_lrs = [group['minimum_lr'] for group
                        in optimizer.param_groups]
        
        self.base_weight_decays = [group['weight_decay'] for group
                                   in optimizer.param_groups]
        
        self.policy = policy
        self.eta_on_restart_cb = eta_on_restart_cb
        self.eta_on_iteration_cb = eta_on_iteration_cb
        if policy_fn is not None:
            self.policy_fn = policy_fn
        elif self.policy == "cosine":
            self.policy_fn = CosinePolicy()
        elif self.policy == "arccosine":
            self.policy_fn = ArccosinePolicy()
        elif self.policy == "triangular":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
        elif self.policy == "triangular2":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
            self.eta_on_restart_cb = ReduceMaxLROnRestart(ratio=0.5)
        elif self.policy == "exp_range":
            self.policy_fn = TriangularPolicy(triangular_step=triangular_step)
            self.eta_on_iteration_cb = ExpReduceMaxLROnIteration(gamma=gamma)
            
        self.last_epoch = last_epoch
        self.batch_size = batch_size
        self.epoch_size = epoch_size
        
        self.iteration = 0
        self.total_iterations = 0
        
        self.t_mult = t_mult
        self.verbose = verbose
        self.restart_period = math.ceil(restart_period)
        self.restarts = 0
        self.t_epoch = -1
        self.epoch = -1
        
        self.eta_min = 0
        self.eta_max = 1
        
        self.end_of_period = False
        self.batch_increments = []
        self._set_batch_increment()
        
    def _on_restart(self):
        if self.eta_on_restart_cb is not None:
            self.eta_min, self.eta_max = self.eta_on_restart_cb(self.eta_min,
                                                                self.eta_max)
            
    def _on_iteration(self):
        if self.eta_on_iteration_cb is not None:
            self.eta_min, self.eta_max = self.eta_on_iteration_cb(self.eta_min,
                                                                  self.eta_max,
                                                                  self.total_iterations)
            
    def get_lr(self, t_cur):
        eta_t = (self.eta_min + (self.eta_max - self.eta_min)
                 * self.policy_fn(t_cur, self.restart_period))
        
        weight_decay_norm_multi = math.sqrt(self.batch_size /
                                            (self.epoch_size *
                                             self.restart_period))
        
        lrs = [min_lr + (base_lr - min_lr) * eta_t for base_lr, min_lr
               in zip(self.base_lrs, self.min_lrs)]
        weight_decays = [base_weight_decay #* eta_t * weight_decay_norm_multi
                         for base_weight_decay in self.base_weight_decays]
        
        if (self.t_epoch + 1) % self.restart_period < self.t_epoch:
            self.end_of_period = True
            
        if self.t_epoch % self.restart_period < self.t_epoch:
            if self.verbose:
                print("Restart {} at epoch {}".format(self.restarts + 1,
                                                      self.last_epoch))
            self.restart_period = math.ceil(self.restart_period * self.t_mult)
            self.restarts += 1
            self.t_epoch = 0
            self._on_restart()
            self.end_of_period = False
            
        return zip(lrs, weight_decays)
        
    def _set_batch_increment(self):
        d, r = divmod(self.epoch_size, self.batch_size)
        batches_in_epoch = d + 2 if r > 0 else d + 1
        self.iteration = 0
        self.batch_increments = torch.linspace(0, 1, batches_in_epoch).tolist()
        
    def step(self):
        self.last_epoch += 1
        self.t_epoch += 1
        self._set_batch_increment()
        self.batch_step()
        
    def batch_step(self):
        try:
            t_cur = self.t_epoch + self.batch_increments[self.iteration]
            self._on_iteration()
            self.iteration += 1
            self.total_iterations += 1
        except (IndexError):
            raise StopIteration("Epoch size and batch size used in the "
                                "training loop and while initializing "
                                "scheduler should be the same.")
        
        for param_group, (lr, weight_decay) in zip(self.optimizer.param_groups,
                                                   self.get_lr(t_cur)):
            param_group['lr'] = lr
            param_group['weight_decay'] = weight_decay

In [ ]:
import os
import os.path as osp
import math

import numpy as np
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T

from torch.utils.checkpoint import checkpoint
from torch_cluster import knn_graph

from torch_geometric.nn import EdgeConv, NNConv
#from torch_geometric.nn.pool.edge_pool import EdgePooling

from torch_geometric.utils import normalized_cut
from torch_geometric.utils import remove_self_loops
from torch_geometric.utils.undirected import to_undirected
from torch_geometric.nn import (graclus, max_pool, max_pool_x,
                                global_mean_pool, global_max_pool,
                                global_add_pool)

transform = T.Cartesian(cat=False)

def normalized_cut_2d(edge_index, pos):
    row, col = edge_index
    edge_attr = torch.norm(pos[row] - pos[col], p=2, dim=1)
    return normalized_cut(edge_index, edge_attr, num_nodes=pos.size(0))

class DynamicReductionNetwork(nn.Module):
    # This model iteratively contracts nearest neighbour graphs 
    # until there is one output node.
    # The latent space trained to group useful features at each level
    # of aggregration.
    # This allows single quantities to be regressed from complex point counts
    # in a location and orientation invariant way.
    # One encoding layer is used to abstract away the input features.
    def __init__(self, input_dim=5, hidden_dim=64, output_dim=1, k=16, aggr='add',
                 #norm=torch.tensor([1./500., 1./500., 1./54., 1/25., 1./1000.])):
                 norm=torch.tensor([1., 1., 1., 1., 1.])):
        super(DynamicReductionNetwork, self).__init__()

        self.datanorm = nn.Parameter(norm,requires_grad=False)
        
        self.k = k
        start_width = 2 * hidden_dim
        middle_width = 3 * hidden_dim // 2

        
        self.inputnet =  nn.Sequential(
            nn.Linear(input_dim, hidden_dim*2),            
            nn.LeakyReLU(negative_slope=0.4),
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.LeakyReLU(negative_slope=0.4),
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.LeakyReLU(negative_slope=0.4),
        )
        
        
        convnn1 = nn.Sequential(nn.Linear(start_width, middle_width),
                                nn.LeakyReLU(negative_slope=0.4),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.LeakyReLU(negative_slope=0.4)
                                )
        convnn2 = nn.Sequential(nn.Linear(start_width*2, middle_width),
                                nn.LeakyReLU(negative_slope=0.4),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.LeakyReLU(negative_slope=0.4)
                                )
        
        convnn3 = nn.Sequential(nn.Linear(start_width*3, middle_width),
                                nn.LeakyReLU(negative_slope=0.4),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.LeakyReLU(negative_slope=0.4)
                                )
        convnn4 = nn.Sequential(nn.Linear(start_width*4, middle_width),
                                nn.LeakyReLU(negative_slope=0.4),
                                nn.Linear(middle_width, hidden_dim),                                             
                                nn.LeakyReLU(negative_slope=0.4)
                                )
                
        self.edgeconv1 = EdgeConv(nn=convnn1, aggr=aggr)
        self.edgeconv2 = EdgeConv(nn=convnn2, aggr=aggr)
        self.edgeconv3 = EdgeConv(nn=convnn3, aggr=aggr)
        self.edgeconv4 = EdgeConv(nn=convnn4, aggr=aggr)
        
        self.shortcut = nn.Linear(input_dim,hidden_dim, bias=False)
        #nn.Conv1d(in_channels, out_channels, kernel_size, stride=1,)
        
        self.output = nn.Sequential(nn.Linear(hidden_dim+2, hidden_dim),
                                    nn.LeakyReLU(negative_slope=0.4),
                                    #nn.Softplus(),
                                    nn.Linear(hidden_dim, (hidden_dim)//2),
                                    nn.LeakyReLU(negative_slope=0.4),
                                    #nn.Softplus(),
#                                    nn.Linear(hidden_dim//2, hidden_dim//2),#added
 #                                   nn.ELU(),
                                    #nn.Softplus(),
                                    nn.Linear((hidden_dim)//2, output_dim)
                                   )
        
        
    def forward(self, data):        
        data.x = self.datanorm * data.x
        #print(data.pscieta.shape)
        ieta = data.pscieta.clone()
        iphi = data.psciphi.clone()
        data.x = self.inputnet(data.x)
        orig_x = data.x.clone()
        #print("orig_x",orig_x.shape)
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv1.flow))
        data.x = self.edgeconv1(data.x, data.edge_index)
        data.x = torch.cat([data.x, orig_x], dim=-1)
        #print("torch.cat([data.x, orig_x], dim=-1",data.x.shape)
        
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data)
        
        res1 = data.x.clone()
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv2.flow))
        data.x = self.edgeconv2(data.x, data.edge_index)
        data.x = torch.cat([data.x, res1], dim=-1)
        #print("torch.cat([data.x, orig_x], dim=-1",data.x.shape)
        
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        data.edge_attr = None
        data = max_pool(cluster, data)
        
        res2 = data.x.clone()
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv3.flow))
        data.x = self.edgeconv3(data.x, data.edge_index)
        data.x = torch.cat([data.x, res2], dim=-1)
        #print("torch.cat([data.x, orig_x], dim=-1",data.x.shape)     
        
        
        data.edge_index = to_undirected(knn_graph(data.x, self.k, data.batch, loop=False, flow=self.edgeconv4.flow))
        data.x = self.edgeconv4(data.x, data.edge_index)
        weight = normalized_cut_2d(data.edge_index, data.x)
        cluster = graclus(data.edge_index, weight, data.x.size(0))
        x, batch = max_pool_x(cluster, data.x, data.batch)

        x = global_max_pool(x, batch)
        #print(x.shape)
        #print(data.pscieta)
        #print(data.iphi.size())
        x = torch.cat([x, ieta, iphi], dim=-1)
#        print(self.output(x))
        return self.output(x).squeeze(-1)

In [ ]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
#from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm as tqdm

#from models.DynamicReductionNetwork import DynamicReductionNetwork

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = DynamicReductionNetwork(input_dim=6,hidden_dim=40,k=16,output_dim=1,norm=torch.tensor([1., 1./64., 1./64., 1/64., 1/64., 1/64.]))
        
    def forward(self, data):
        logits = self.drn(data)
        #return F.softplus(logits)
        return logits
device = torch.device('cuda:1')#('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
scheduler = CyclicLRWithRestarts(optimizer, ntrainbatch, epoch_size, restart_period=80, t_mult=1.2, policy="cosine")
#criterion = torch.nn.MSELoss()

#lossmse = nn.MSELoss()
def resoloss2(output,truth):
    batch_size = output.size()[0]
    #mse = F.mse_loss(output, truth, reduction='mean')
    #mse = torch.sum((output-truth)**2/truth)/batch_size
    #mse = torch.sum(((output-truth)/truth)**2)/batch_size
    mse = torch.mean(torch.abs((output - truth)))
    
    #res = 
    return (mse)

def resoloss3(output,truth):
    #batch_size = output.size()[0]
    #mse = F.mse_loss(output, truth, reduction='mean')
    #mse = torch.sum((output-truth)**2/truth)/batch_size
    #mse = torch.sum(((output-truth)/truth)**2)/batch_size
    #mse = torch.mean(torch.abs((output - truth)))
    #outputa = output[truth < 1.0]
    #outputa = output[truth > 1.0]
    loss1 = torch.mean(torch.abs((output[truth < 1.0] - truth[truth < 1.0])))
    batch_size = len(output[truth > 1.0])
    #loss2 = torch.sum((output[truth > 1.0] -truth[truth > 1.0] )**2/truth[truth > 1.0] )/batch_size
    loss2 = torch.sum((output[truth > 1.0] -truth[truth > 1.0] )**2 )/batch_size

    #res = 
    return loss1+loss2
    

    
def resoloss(outputa,trutha):
    n_bins = np.random.poisson(20) + 2
    minx = torch.min(trutha).detach().cpu()
    maxx = torch.max(trutha).detach().cpu()
    minx = np.exp(minx)
    maxx = np.exp(maxx)
    
    binss = np.arange(minx,maxx,(maxx-minx)/n_bins)
    binss = np.log(binss)
    loss = 0.
    for i in range(binss.size-1):
        tmpo = outputa[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
        tmpt = trutha[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
        #print("tmpo,tmpt:",tmpo,tmpt)
        if (list(tmpo.size())[0] == 0 or list(tmpt.size())[0] == 0):
            continue
        #print(list(tmpo.size())[0],list(tmpt.size())[0])
        tmpom = torch.mean(tmpo)
        tmptm = torch.mean(tmpt)
        #print (tmpom,tmptm)
        #if (tmptm == 0):
        #    continue
        
        #    if ((tmpom - tmptm)**2/tmptm is not np.nan):
        #        print ("(tmpom - tmptm)**2/tmptm",(tmpom - tmptm)**2/tmptm)
        loss += (tmpom - tmptm)**2/tmptm
    
    
    if (loss == 0.):
        print("minx,maxx,binss,n_bins:",minx,maxx,binss,n_bins)    
        print(outputa,trutha)
        for i in range(binss.size-1):
            tmpo = outputa[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
            tmpt = trutha[ (trutha > binss[i]) &  (trutha < binss[i+1])  ]
            print("tmpo,tmpt:",tmpo,tmpt)
            if (list(tmpo.size())[0] == 0 or list(tmpt.size())[0] == 0):
                continue
            #print(list(tmpo.size())[0],list(tmpt.size())[0])
            tmpom = torch.mean(tmpo)
            tmptm = torch.mean(tmpt)
            print (tmpom,tmptm)
            loss += (tmpom - tmptm)**2/tmptm
            print(loss)
        
    #print("loss: ",loss)
    return loss        
losscat = torch.nn.CrossEntropyLoss()    
def categorical_loss(outputa,trutha,alpha):
    total_loss =  losscat(outputa[:,:2],trutha[:,0].long()) + alpha*resoloss(outputa[:,2],trutha[:,1])
    return total_loss


def categorical_loss_only(outputa,trutha):
    total_loss =  losscat(outputa[:,:2],trutha[:,0].long()) 
    return total_loss


#model.train()
def train(epoch):
    model.train()
    torch.cuda.empty_cache()
    scheduler.step()
    loss = []
    for data in tqdm(trainloader):
            #print("\n\nScaled data: ")
            #print(data.X[:,0].flatten())
            #print("\n Unscaled data: ")
            #print(data.x[:,0].flatten())
            data = data.to(device)
            #print("\n train data= ", data)
            optimizer.zero_grad()
            result = inv_transform(model(data))
            truthv = inv_transform(data.genmass.flatten())
            #print("train result: ",result)
            #print("original mass: ",truthv)
            lossc = resoloss2(result, truthv)
            #print("result:",result)
            #print("truth:",data.z)
            #lossc = categorical_loss(result, data.z, 1.0)
#            lossc = categorical_loss_only(result, data.z)
#            mse = F.mse_loss(result, data.y, reduction='mean')
#            mse = criterion(result, data.y)
#            print('result, y:',result,data.y)
#            print('crit, orig:',criterion(result, data.y),F.mse_loss(result, data.y, reduction='mean'))
            loss.append(lossc.item()) 
            lossc.backward()
#            print(mse)
            optimizer.step()
            scheduler.batch_step()
    print( 'batches for train:',len(loss)) 
#    print('loss',loss)
    print('train loss:',np.mean(np.array(loss)))
    return np.mean(np.array(loss))
    #del loss #memtest
#    print(data)

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as scs
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
#%matplotlib inline

def gaussian(x,  mean,a, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))

def evaluate(epoch):
        """"Evaluate the model"""
        model.zero_grad()
        torch.cuda.empty_cache()
        model.eval()
#        loss = []
#        frac = []

        pred = []
        true = []
        loss= []
        
        correct = 0
        predc = []
        truec = []
        for data in tqdm(testloader):
            data = data.to(device)        
            result = inv_transform(model(data))
            truthv = inv_transform(data.genmass.flatten())
            #print("eval result: ",result)
            #print("original mass: ",truthv)
            lossc = resoloss2(result, truthv)
#            lossc = resoloss(result, data.genmass)
#            lossc = categorical_loss_only(result, data.z)
#            print (result.item(),data.y.item())
#            frac.append((result.item() - data.y.item())/data.y.item())
            loss.append(lossc.item())

            for i in result:
                pred.append(i.detach().cpu())
                #predc.append(i.detach().cpu().argmax())
            for i in truthv.detach():
                true.append(i.detach().cpu())
                #truec.append(i.detach().cpu())
            
            
        #print(predc,truec)
        #predc = np.array(predc)
        #truec = np.array(truec)
        #print("accuracy  :",np.equal(predc,truec).sum()/len(truec))
        
        print('batches for test:', len(loss)) 
        print('test loss:',np.mean(np.array(loss)))
#        fracarr = np.array(frac

        
        preda = np.array(pred)
        truea = np.array(true)
        #preda = preda[:,2] ### added
        #truea = truea[:,1] ### added
        fracarr = (preda - truea)/truea
        #print(preda,truea,fracarr)
        print('pred - true / true mean:',(np.mean(fracarr)))
        print('pred - true / true std:',(np.std(fracarr)))
        (mu, sigma) = norm.fit(fracarr)
        print('mu,sig:',mu,sigma)
        

        #bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=100, label='histogram')
        #bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
        
        '''from matplotlib.pyplot import figure
        figure(figsize=(30, 20), dpi=40)
        plt.rcParams['axes.labelsize'] = 36
        plt.rcParams['axes.titlesize'] = 36
        plt.hist(fracarr,bins=100,range=[-4,4])
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.axvline(x=0.0,c='r')
        plt.show()'''
        
        
        bins =  np.linspace(0,18,19)

        fig, axs = plt.subplots(6,3, figsize=(40, 40), facecolor='w', edgecolor='k')
        axs = axs.ravel()

        for i in tqdm(range (bins.size - 1)):
            predaa = preda[(truea >bins[i]) & (truea <bins[i+1]) ]
            trueaa = truea[(truea >bins[i]) & (truea <bins[i+1]) ]
            fracarr = (predaa - trueaa)/trueaa
            #if (fracarr < 0):
            axs[i].hist(fracarr,bins=100,range=[-4,4],)
            axs[i].set_xlabel('pred - true / true')
            axs[i].set_ylabel('counts')
            axs[i].set_title(str(bins[i])+" to "+str(bins[i+1]))
            #print (vals)
        if (epoch%5==0):
            plt.savefig('%s/mt-mp_frac_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()
        
        '''try:
            from matplotlib.pyplot import figure
            figure(figsize=(20, 10), dpi=30)
            plt.rcParams['axes.labelsize'] = 16
            plt.rcParams['axes.titlesize'] = 16
            popt, _ = curve_fit(gaussian, bin_centers, bin_heights, p0=[0., 100., 1.],bounds = ([-np.inf,0,0],[np.inf,np.inf,np.inf]))
            x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 100)
            plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fit')
            plt.legend()


            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.yscale("log")
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(popt[0], popt[2]))
            plt.grid(True)

            plt.show()

        except RuntimeError:
            print("Error - curve_fit failed")
            plt.xlabel('pred - true / true')
            plt.ylabel('counts')
            #plt.title(r'$\mathrm{pred - true / true:}\ \mu=%.3f,\ \sigma=%.3f$' %(mu, sigma))
            plt.title('pred - true / true fit failed')
            #plt.yscale("log")
            plt.grid(True)

            plt.show()'''
        from matplotlib.pyplot import figure
        figure(figsize=(30, 30), dpi=40)
        #plt.rcParams['axes.labelsize'] = 36
        #plt.rcParams['axes.titlesize'] = 36
        #plt.hist2d(truea,preda,bins=200)
        plt.scatter(truea,preda,alpha=0.4)
        plt.plot([-4,18], [-4,18], 'k-')
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.xlim([-2, 18])
        plt.ylim([-2, 20])
        plt.axvline(x=1.0,c='r')
        plt.axhline(y=1.0,c='r')
        #plt.show()
        if (epoch%5 ==0):
            plt.savefig('%s/mpredVsmtrue_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()
        return np.mean(np.array(loss))
        #del pred,true,loss,preda,truea,fracarr  #memtest

In [ ]:
#from tqdm import notebook.tqdm as tqdm
#import tqdm.notebook.tqdm as tqdm
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_testtesttesttest_dyn2addlayer/'
#checkpoint_dir = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_add1ip1dyn_binresoloss/'
#"/home/llr/cms/sghosh/GNNECAL/model_nopho_deepernet/"
nepoch=500
os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
losst = []
lossv = []
epochs = []
for epoch in range(nepoch):
    print ('epoch:',epoch)
    losst.append(train(epoch))
    loss_epoch = evaluate(epoch)
    lossv.append(loss_epoch)
    epochs.append(epoch)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    checkpoint_file = 'model_epoch_%i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_checkpoint_best.pth.tar' ))
    if ((epoch+1)%10 == 0):
        plt.plot(np.array(epochs),np.array(losst),c='b',label='training')
        plt.plot(np.array(epochs),np.array(lossv),c='r',label='testing')
        plt.legend()
        if ((epoch+1) == nepoch):
            plt.savefig('%s/TrainvsValLoss_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()


In [ ]:
device = torch.device('cpu')#('cuda' if torch.cuda.is_available() else 'cpu')
#checkpoint_dir = '/export/home/phys/aharilal/MassRegressor/a2gg-regression/MODELS/GNN_scale50/'
ep_test=299
model_fname = checkpoint_dir + '/model_epoch_%d.pth.tar'%ep_test
model= Net().to(device)
model.load_state_dict(torch.load(model_fname)['state_dict'])
model.eval()

In [ ]:
#model_fname = '/home/sameasy2006/hgcal_ldrd-gravnet2_wip_trainer_args/ouput_regression_phopi_comb_categ/model_cont3_checkpoint_best.pth.tar'
#model_fname = '/home/llr/cms/sghosh/HGCAL_TICL_STUFF/RECOTESTS_ORGANISED/MODTICLLXPLUS/CMSSW_11_2_0_pre10/src/RecoNtuples/HGCalAnalysis/test/GENGRAPHS/ouput_classification_PU_LC3213/model_checkpoint_best.pth.tar'
#checkpoint_dir = '/home/llr/cms/sghosh/HGCAL_TICL_STUFF/RECOTESTS_ORGANISED/MODTICLLXPLUS/CMSSW_11_2_0_pre10/src/RecoNtuples/HGCalAnalysis/test/GENGRAPHS/ouput_PU_morestats_ptweight/'

#model_fname = checkpoint_dir + 'model_checkpoint_best.pth.tar'
model_fname = checkpoint_dir + 'model_epoch_299.pth.tar'
mdl=Net().to(device)
mdl.load_state_dict(torch.load(model_fname)['state_dict'])
mdl.eval()
print(sum(p.numel() for p in model.parameters()))

In [ ]:
loss_epoch = evaluate(epoch)